In [23]:
# imports
import os
import h5py
import xml.etree.ElementTree as ET
import pathlib
import numpy as np
import numpy.fft
import matplotlib.pyplot as plt

In [31]:
def histeq(im,nbr_bins=256):
  """  Histogram equalization of a grayscale image. """

  # get image histogram
  imhist,bins = np.histogram(im.flatten(),nbr_bins,normed=True)
  cdf = imhist.cumsum() # cumulative distribution function
  cdf = 255 * cdf / cdf[-1] # normalize

  # use linear interpolation of cdf to find new pixel values
  im2 = np.interp(im.flatten(),bins[:-1],cdf)

  return im2.reshape(im.shape)

In [3]:
input_data_path = '/mnt/data/singlecoil_train/'
output_data_path = '/mnt/data_2/singlecoil_train/'

In [4]:
files = [os.path.join(input_data_path, f) for f in os.listdir(input_data_path)]
path = pathlib.Path(output_data_path)
path.parent.mkdir(parents=True, exist_ok=True)

In [38]:
ven_model_count = {}
for fp in files:
    f = h5py.File(fp)
    
    xml_doc = f.get("ismrmrd_header")[()]
    root = ET.fromstring(f.get('ismrmrd_header')[()])
    ns = {'ns': 'http://www.ismrm.org/ISMRMRD'}
    acq_sys_info = root.findall("ns:acquisitionSystemInformation", ns)[0]
    
    Vendor = acq_sys_info.find("ns:systemVendor", ns).text
    Model = acq_sys_info.find("ns:systemModel", ns).text
    if acq_sys_info.find("systemFieldStrength_T", ns) != None:
        T = acq_sys_info.find("systemFieldStrength_T", ns).text
    else:
        T = "NA"
    
    Key = Vendor + "-" + Model + "-" + T
    
    if Key in ven_model_count:
        ven_model_count[Key] = ven_model_count[Key] + 1
    else:
        ven_model_count[Key] = 1
        
    if Key == 'SIEMENS-Biograph_mMR-NA':
        fn = output_data_path + "data-" + str(ven_model_count[Key]) +  ".npy";
        
        k = f.get("kspace")
    
        img = np.abs(np.fft.fftshift(np.fft.ifft2(k)))
    
        min_x = int(img.shape[1]/2 - 160)
        max_x = int(img.shape[1]/2 + 160)
        min_y = int(img.shape[2]/2 - 160)
        max_y = int(img.shape[2]/2 + 160)
    
        img = img[:,min_x:max_x,min_y:max_y]
    
        row_sums = img.sum(axis=2)
        norm_img = img / row_sums[:, numpy.newaxis]
        
        
        np.save(fn, norm_img)
    
print(ven_model_count)

{'SIEMENS-Skyra-NA': 417, 'SIEMENS-Aera-NA': 411, 'SIEMENS-Biograph_mMR-NA': 104, 'SIEMENS-Prisma_fit-NA': 41}
